In [1]:
import pandas as pd
import numpy as np
import taxopy
import glob
import sys 
import os

In [2]:
#Functions to get catalogs and mathces IDs

def read_matches(matches_file):
    '''
    This function reads the .matches files of sstacks files as dataframes returning the catalog and sample ID
    '''
    
    
    matches = pd.read_csv(matches_file,sep='\t',
                          on_bad_lines='skip', 
                          skiprows=1 ,header=None, 
                          names = ['loci_id','sample_id','locus_id','h','depth','cigar'],
                          usecols=['loci_id','sample_id'],
                         dtype = {'loci_id':'str', 'sample_id':'str'})
    matches = matches[:-1]
    matches = matches.astype({'loci_id':'str'})
    
    return matches


def read_blast_results(path_to_blast_results):
    '''
    This function reads blast results in csv format. 
    csv file must have the next structure
    
    Query ID    |Subject ID    |TaxonomicID    |Description    |Identity%    |Number of mismatches
    
    The function read the complete table, but return just mathces with the highest percentage of identity per Query
    in a dataframe
    '''
    #read blast resutls in csv format
    blast_results = pd.DataFrame()
    for file in os.listdir(path_to_blast_results):
        if '.gz' in file:
            tmp = pd.read_csv(path_to_blast_results + '/' + file, sep = '\s+', compression='gzip', 
                          names=['queryID','subjectID','taxID','Description','%id','mismatchess'],
                          dtype={'queryID':str,'subjectID':str,'taxID':str,'Description':str,'%id':float,'mismatchess':int})
            blast_results = pd.concat([blast_results,tmp])
    
    # re index the matrix
    blast_results = blast_results.reset_index(drop=True)
    
    # get maximun % identity matches
    max_index = blast_results.groupby('queryID')['%id'].idxmax()
    tmp_blast = blast_results.loc[max_index]
    
    return tmp_blast

def get_tax_information(blast_results):
    '''
    This function allow to extract the taxonomic ID of blast results dataframe using taxopy package.
    The function read the ID, and get the complete taxID. However, we just utilize principal ranks because not
    all entries in TaxDB has a complete lineage (i.e H. sapiens has 30 ranks while E. coli just 7)
    '''
    
    
    #extrac uniques tax ids

    taxIDs = blast_results['taxID'].drop_duplicates()

    lineages = []
    lin = ['superkingdom','phylum','class','order','family','genus','species']

    for tax_id in taxIDs:
        
        #get the taxon
        #in some cases, we can have more than one tax ID, in ths format, ID;ID;ID, we choose the first one
        tax_id = tax_id.split(';')[0]
        lineage = taxopy.Taxon(int(tax_id), taxdb)
    
        #convert taxon into a dictionary
        lineage = lineage.rank_name_dictionary
    
        # get only specific ranks if they exist in the dictionary
        lineage = [lineage.get(key,np.nan) for key in lin]
    
        #append the taxID to the begining, this allow us to convert easly the complete lineeage into a dataframe
        lineage.insert(0,tax_id)
        lineages.append(lineage)
        
        # convert the lienages lists, into a dataframe
    lineages = pd.DataFrame(lineages,
                       columns = ['taxID'] + lin)
    return lineages

def merge_tax_matches(macthes,lineages, blast_df):
    '''
    This function allow to merge all dataframes: matches, blast, lineages
    '''
    
    #merge matches, with blast results
    matches_blast = pd.merge(macthes, blast_df, left_on='loci_id', right_on='queryID')

    #merge matches, blast table with lineage information
    matches_blast_tax = pd.merge(matches_blast, lineages, on = 'taxID').drop_duplicates()
    
    return matches_blast_tax

def read_pop_map(popmap):
    '''
    This function allow to read popmap file that will be mapped with the merged table
    '''
    popdict = pd.read_csv(popmap,sep = '\t',header = None,
                          names=['sample','pop'],
                          index_col='sample').T.to_dict('records')[0]
    
    return popdict

In [ ]:
#this code block needs to run one time, load taxonomi DB, and blast results

taxdb = taxopy.TaxDb()   # this can take a while. If crash, try again

path_to_blast_results = '/home/alexsanyum/meta_project/server_results/blast_analyses/results3'
path_to_mathces = 'tepui_stacks/'
pop_map = '~/meta_project/popmap.txt'

popdict = read_pop_map(pop_map)
blast_results = read_blast_results(path_to_blast_results) 
tax_info = get_tax_information(blast_results)


In [8]:
# iterate over all mathces files 

all_matches = []
log_match = '{0}\t{1}\t{2}\t{3} %\n'.format('Sample','Total Matches','Found in blast','Percentage of found')
for file in os.listdir(path_to_mathces):
    if 'matches.tsv.gz' in file:
        #read matches table
        matches = read_matches(path_to_mathces + file)
        matches['loci_id'] = 'catID:' + matches['loci_id']
        
        #merge with blast, and tax information
        merge_matches = merge_tax_matches(matches,tax_info,blast_results)
        
        #add sample and population information
        sample =  file.split('.')[0]
        population = popdict[sample]
        
        merge_matches['Sample'] = sample
        merge_matches['Population'] = population
        
        all_matches.append(merge_matches)
        
        log_match += '{0}\t{1}\t{2}\t{3:.2f} %\n'.format(sample, 
                                                       len(matches),
                                                       len(merge_matches),
                                                       len(merge_matches) * 100 /len(matches) ) 
all_matches = pd.concat(all_matches)
all_matches.iloc[:,2:].to_csv('blast_matches.csv',sep =',')

with open('match_log.txt','w') as f:
    f.write(log_match)
        

In [9]:
# remove reads that matched with Amphibia 

masks = (all_matches['class'] != 'Amphibia')

filterd_match = all_matches[masks]

#Remove genus that appears just one time in all the samples

#count the number of occurences in genus count
species_count = all_matches['genus'].value_counts()

#get the index of species that appears more than one time
species_to_keep = species_count[species_count >1].index

#remove
filterd_match = filterd_match[filterd_match['genus'].isin(species_to_keep)]


In [18]:
#Write csv file 

filterd_match = filterd_match.drop(['loci_id','sample_id','Description'],axis=1)
filterd_match.to_csv('no_unique_genus_matches.csv',sep = ',')

In [19]:
filterd_match

,queryID,subjectID,taxID,%id,mismatchess,superkingdom,phylum,class,order,family,genus,species,Sample,Population
0,catID:551553,gi|1559969735|gb|CP032595.1|,315492,92.958,3,Eukaryota,Chordata,Actinopteri,Acropomatiformes,Lateolabracidae,Lateolabrax,Lateolabrax maculatus,Ab_403_GAGTC-ATCACG,2
1,catID:103562,gi|1768674649|emb|LR722623.1|,112509,100.000,0,Eukaryota,Streptophyta,Magnoliopsida,Poales,Poaceae,Hordeum,Hordeum vulgare,Ab_403_GAGTC-ATCACG,2
2,catID:596882,gi|1905168431|emb|LR880655.1|,8081,98.734,1,Eukaryota,Chordata,Actinopteri,Cyprinodontiformes,Poeciliidae,Poecilia,Poecilia reticulata,Ab_403_GAGTC-ATCACG,2
0,catID:192241,gi|1695378300|emb|LR597471.1|,375764,100.000,0,Eukaryota,Chordata,Actinopteri,Kurtiformes,Apogonidae,Sphaeramia,Sphaeramia orbicularis,Er_462_GTCGA-CGATGT,4
1,catID:332112,gi|1695378314|emb|LR597475.1|,375764,68.750,12,Eukaryota,Chordata,Actinopteri,Kurtiformes,Apogonidae,Sphaeramia,Sphaeramia orbicularis,Er_462_GTCGA-CGATGT,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24779,catID:325389,gi|1537846887|gb|CP034454.1|,2493679,88.372,5,Bacteria,Pseudomonadota,Alphaproteobacteria,Hyphomicrobiales,Phyllobacteriaceae,Mesorhizobium,Mesorhizobium sp. M8A.F.Ca.ET.057.01.1.1,Er_R-06_AACCA-CGATGT,5
24780,catID:292687,gi|2249649983|gb|CP094634.1|,883000,92.105,3,Eukaryota,Streptophyta,Magnoliopsida,Brassicales,Brassicaceae,Camelina,Camelina hispida,Er_R-06_AACCA-CGATGT,5
24782,catID:174765,gi|2423789207|gb|MZ502680.1|,681932,87.640,11,Eukaryota,Chordata,Actinopteri,Characiformes,Characidae,Pristella,Pristella maxillaris,Er_R-06_AACCA-CGATGT,5
24783,catID:585518,gi|1575822791|gb|CP035765.1|,13689,91.358,7,Bacteria,Pseudomonadota,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Sphingomonas,Sphingomonas paucimobilis,Er_R-06_AACCA-CGATGT,5
